# FOUNDATION OF COMPUTER SCIENCE PROJECT


Vittoria Porta (861784)

Il progetto è stato realizzato utilizzando i dati provenenienti dal database Kiva  (https://www.kaggle.com/gaborfodor/additional-kiva-snapshot) che fornisce informazioni su circa 1,4 milioni di prestiti e 2,3 milioni di prestatori che provengono da tutto il mondo.  Complessivamente il database contiene 7 files (loans.csv ,lenders.csv, loans_lenders.csv, country_stats.csv, GEconV4.csv, locations.csv, loan_coords.csv).


Le librerie che mi sono state utili per svolgere il lavoro vengono riportate di seguito:

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

%load_ext autotime

In [2]:
#pip install ipython-autotime

time: 597 µs


Utilizzo questa funzione per il calcolo totale del tempo di esecuzione di tutte le celle

In [3]:
initial= dt.datetime.now()

time: 1.15 ms


# QUESTION 1)
- Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

Per prima cosa ho importato la tabella loans_lenders, con l'utilizzo della libreria pandas.

In [4]:
loans_lenders = pd.read_csv(r'./cs/loans_lenders.csv')

time: 5.47 s


In [5]:
#loans_lenders.head(5)

time: 633 µs


La prima domanda chiede di ristrutturare il dataframe in maniera da ottenere su ciascuna riga un solo codice del prestito 'loan_id' e un solo prestatore 'lender': in questo modo ho deciso di creare una lista vuota che viene riempita tramite un doppio ciclo 'for' che separa la colonna non strutturata dei lenders (con il separatore (', '), così da ottenere una lista di lenders per ogni 'loan_id' e i 'loan_id' vengono ripetuti tante volte quanti sono i lenders).

Il risultato ottenuto è un nuovo dataframe (norm_loans_lenders) contenente 28293931 righe, ciascuna delle quali composta da un lender e dal loan_id corrispondente al prestito.

In [6]:
loans_list = []
for i, j in zip(loans_lenders.loan_id, loans_lenders.lenders):
    for el in j.split(', '):
        loans_list.append([i] + [el])

norm_loans_lenders = pd.DataFrame(loans_list,columns = ['loan_id','lenders'])
norm_loans_lenders.head(10)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
5,483693,karlheinz4543
6,483693,jerrydb
7,483693,paula8951
8,483693,gmct
9,483693,amra9383


time: 50.8 s


# QUESTION 2)
- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Per rispondere al secondo quesito ho, per prima cosa caricato (utilizzando nuovamente la libreria pandas), il file loans.csv

In [7]:
loans_csv = pd.read_csv(r'./cs/loans.csv')

time: 1min


Per prima è stato necessario effettuare una fase di esplorazione del dataset per conoscere la natura delle variabili.


In [8]:
#loans_csv.columns
#loans_csv.dtypes

time: 1.27 ms


Grazie alla prima fase di esplorazione, ho notato che entrambe le date utili per la risoluzione del quesito erano 'objects': le ho dunque trasformate in formato datetime tramite la funzione 'pandas.to_datetime'.

Secondariamente, ho aggiunto una nuova colonna al dataframe già esistente 'loans_csv' chiamata 'duration' e ho eseguito una sottrazione tra le due date utilizzando la funzione pandas.Series.dt.days che calcola il numero di giorni per ogni elemento.

In [9]:
data1 = pd.to_datetime(loans_csv["disburse_time"])
data2 = pd.to_datetime(loans_csv["planned_expiration_time"])
loans_csv['duration'] = (data2 - data1).dt.days

time: 1.37 s


Dopo di che ho creato un nuovo dataframe che mostrasse i risultati ottenuti aggiungendo la colonna duration (n. di giorni).

In [10]:
df1 = pd.DataFrame(columns=['Loan ID','Disburse Time','P_Expiration','duration'])
df1['Loan ID'] = loans_csv['loan_id']
df1['Disburse Time'] = loans_csv['disburse_time']
df1['P_Expiration'] = loans_csv['planned_expiration_time']
df1['duration'] = loans_csv['duration']
df1.head(10)

,Loan ID,Disburse Time,P_Expiration,duration
0,657307,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000,53.0
1,657259,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000,96.0
2,658010,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000,37.0
3,659347,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000,34.0
4,656933,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000,57.0
5,659605,2014-01-15 08:00:00.000 +0000,2014-02-20 02:30:07.000 +0000,35.0
6,660240,2014-01-20 08:00:00.000 +0000,2014-02-21 07:50:11.000 +0000,31.0
7,661601,2014-01-10 08:00:00.000 +0000,2014-02-25 09:50:03.000 +0000,46.0
8,661960,2014-01-23 08:00:00.000 +0000,2014-02-26 00:10:03.000 +0000,33.0
9,662117,2014-01-10 08:00:00.000 +0000,2014-03-19 13:40:02.000 +0000,68.0


time: 651 ms


All'interno della colonna 'duration' ci sono 14395 valori negativi e 374645 missing values

In [11]:
#print(len(df1[df1['duration']<0]))  = 14935   
#print(df1['duration'].isnull().sum()) = 374645

time: 870 µs


# QUESTION 12)
- For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must 
have its amount distributed proportionally tothe number of days in each year. For example, a loan with disburse time December 1st, 2016, 
planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365/ (31+365+30) = 4284.04 for 2017,and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Ho deciso di trasferire il punto 12) dopo il punto 2) dal momento che ho tutte le informazioni disponibili a svolgerlo; così facendo lascerò i punti che riguardano l'analisi delle nazioni nei punti a seguire.

Per eseguire il punto 12) decido in primo luogo di creare 6 nuove variabili che vengono aggiunte al dataframe 'loans_csv' che rispettivamente estraggono gli anni, i mesi e i giorni delle date 'disburse_time' e 'planned_expiration_time' (tramite la funzione di pandas dt.year, dt.month e dt.day).

Ho inteso come 'disburse_time' la data di inizio del prestito e come 'planned_expiration_time' la data di estinzione del prestito.

In [12]:
loans_csv['year_plan'] = pd.DataFrame(data2.dt.year) 
loans_csv['year_disbu'] = pd.DataFrame(data1.dt.year)      
loans_csv['day_plan'] = pd.DataFrame(data2.dt.day) 
loans_csv['day_disbu'] = pd.DataFrame(data1.dt.day)
loans_csv['month_plan'] = pd.DataFrame(data2.dt.month) 
loans_csv['month_disbu'] = pd.DataFrame(data1.dt.month)    

time: 546 ms


Ho creato un successivo dataframe 'tot_loan' che comprendesse tutte le informazioni necessarie allo svolgimento dell'esercizio.
Ho recuperato e aggiunto la variabile 'duration' precedentemente calcolata al punto 2) che mi calcola la differenza in giorni tra 'disburse_time' e 'planned_expiration_time' (ora rinominata in 'tot_days' e ho aggiunto la variabile 'debitxday' che calcola l'amount giornaliero dividendo l'amount totale fratto i giorni totali tra una e l'altra data.

In [13]:
tot_loan = pd.DataFrame(columns = ['loan_id','disburse_time','planned_expiration_time','loan_amount','year_plan','year_disbu','month_plan','month_disbu','day_plan','day_disbu','tot_days'])
tot_loan['loan_id'] = loans_csv['loan_id']
tot_loan['disburse_time'] = data1
tot_loan['planned_expiration_time'] = data2
tot_loan['loan_amount'] = loans_csv['loan_amount']
tot_loan['year_plan'] = loans_csv['year_plan']
tot_loan['year_disbu'] = loans_csv['year_disbu']
tot_loan['month_plan'] = loans_csv['month_plan']
tot_loan['month_disbu'] = loans_csv['month_disbu']
tot_loan['day_plan'] = loans_csv['day_plan']
tot_loan['day_disbu'] = loans_csv['day_disbu']
tot_loan['tot_days'] = loans_csv['duration']
tot_loan['debitxday'] = tot_loan['loan_amount'] / tot_loan['tot_days']
tot_loan.reset_index()
tot_loan.head(5)
#tot_loan.dtypes

,loan_id,disburse_time,planned_expiration_time,loan_amount,year_plan,year_disbu,month_plan,month_disbu,day_plan,day_disbu,tot_days,debitxday
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2014.0,2013.0,2.0,12.0,14.0,22.0,53.0,2.358491
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2014.0,2013.0,3.0,12.0,26.0,20.0,96.0,4.166667
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,2014.0,2014.0,2.0,1.0,15.0,9.0,37.0,10.810811
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,2014.0,2014.0,2.0,1.0,21.0,17.0,34.0,18.382353
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2014.0,2013.0,2.0,12.0,13.0,17.0,57.0,7.456140


time: 3.29 s


La question mi richiede di visualizzare due colonne: gli anni e l'amount pagato in quell'anno. 
Per arrivare a fare ciò ho prima di tutto capito quali sono gli anni che compaiono nella data 'disburse_time'.

In [14]:
print(data1.dt.year.unique()) 


[2013. 2014. 2015. 2012. 2016. 2010. 2017. 2018. 2009. 2011. 2007.   nan
 2006. 2008. 2005.]
time: 137 ms


In [15]:
#any(tot_loan['tot_days']>365)= TRUE -------> ho capito che ci sono delle date che 
                                              #iniziano in un anno e finiscono in anni successivi e quindi la differenza 
                                              #tra loro è maggiore di 365 giorni totali.
#min(tot_loan['tot_days'])= -138 -----------> ci sono delle date negative
#max(tot_loan['tot_days'])= 1673 -----------> il massimo di differenza tra le due date è di circa 4 anni e 6 mesi

time: 744 µs


Ho creato un dizionario vuoto associato agli anni con range (2005:2018)

In [16]:
results={2005:0,2006:0,2007:0,2008:0,2009:0,2010:0,2011:0,2012:0,
   2013:0,2014:0,2015:0,2016:0,2017:0,2018:0}


time: 1.79 ms


Prima di riempire il dizionario con i dati finali, ho esplorato il dataset riscontrando alcuni problemi:
ad esempio, qui di seguito viene riportato un ciclo che mi ha mostrato che tutte le 203 righe dell'anno 2005 hanno data di fine 'planned_expiration_time' mancante e data di inizio 'disburse_time' identiche:
'2005-04-14 05:27:55+00:00'.

Probabilmente si è di fronte ad un errore di imputazione o raccolta dati.

In [17]:
#PROBLEMA NELL'ANNO 2005
#count = 0
#for i in range (len(tot_loan['loan_id'])):
    #if tot_loan['year_disbu'][i]==2005:
      #print(tot_loan['disburse_time'][i],tot_loan['planned_expiration_time'][i])
       #count= count+1
        
        
        
#count = 203 

time: 616 µs


Dopo aver riscontrato questo problema ho voluto controllare quante fossero in totale le righe con date mancanti e ho riscontrato che 
- le righe senza 'disburse_time' sono : 2811
- le righe senza 'planned_expiration_time' sono : 371832
- le righe senza entrambe le date sono solo 2
- non esistono righe in cui 'loan_amount' sia nullo

In [18]:
#count_no_inizio=0
#count_no_fine=0
#count_nessuna=0
#tot_amount_null=0
#for i in range (len(tot_loan['loan_id'])):
#   if np.isnan(tot_loan['year_disbu'][i]) and not(np.isnan(tot_loan['year_plan'][i])):
#       count_no_inizio+=1
#    if np.isnan(tot_loan['year_plan'][i]) and not(np.isnan(tot_loan['year_disbu'][i])):
#        count_no_fine+=1
#    if np.isnan(tot_loan['year_disbu'][i]) and np.isnan(tot_loan['year_plan'][i]):
#        count_nessuna+=1
#    if np.isnan(tot_loan['loan_amount'][i]):
#        tot_amount_nul +=1

time: 793 µs


In [19]:
#print(count_no_inizio)= 2811
#print(count_no_fine)= 371832
#print(count_nessuna)=2
#print(tot_amount_null)=0

time: 606 µs


Ho creato una funzione 'daystoend' che prende una data come parametro e calcola i giorni trascorsi dalla data inserita a fine anno. E' utile per la suddivisione degli amount quando l'anno della data 'disburse_time' e l'anno della data 'planned_expiration_time' non coincidono.

In [20]:
def daystoend (date, format='%Y-%m-%d'):
    date = pd.to_datetime(date, format=format)
    last_day = pd.Timestamp(year=date.year, month=12, day=31)
    return (-1) * ((date - last_day).days + 1)

time: 2.84 ms


In [21]:
for i in range (0, len(tot_loan['loan_id'])):
    start_year = tot_loan['year_disbu'][i]      #ho rinominato le variabili così da poterle richiamare più velocemente all'interno del ciclo
    end_year = tot_loan['year_plan'][i]    
    tot_amount = tot_loan['loan_amount'][i]
    
    #con il primo if verifico i casi in cui l'anno di inizio 'disburse_time' esiste 
    #ma non l'anno di fine. Se ciò si verifica tutto l'amount viene destinato all'anno 
    #di inizio   
    if np.isnan(end_year) and not(np.isnan(start_year)): 
        results[start_year] += round(tot_amount,2)
        
    #la seconda condizione verifica i casi in cui l'anno di inizio 'disburse_time' non esiste 
    #ma è presente l'anno di fine. Se ciò si verifica tutto l'amount viene destinato all'anno 
    #di fine. 
    elif np.isnan(start_year) and not(np.isnan(end_year)):
        results[end_year] += round(tot_amount,2)
   
    else:
        diff_year = end_year - start_year               #la variabile 'diff_year' calcola la differenza tra l'anno di fine e quello di inizio
        diff_year = diff_year.astype(np.int64)

        #la prima condizione verifica i casi in cui la differenza di anni sia uguale a 0, quindi l'anno di inizio e fine coincidono. 
        #Perciò l'intero amount viene inserito nell'anno corrispondente di inzio.
        if diff_year == 0:
            results[start_year] += round(tot_amount,2)
            
        #la seconda condizione verifica i casi in cui la differenza di anni sia maggiore di 0, quindi l'anno di inizio e fine non coincidono. 
        #Perciò l'intero amount viene suddiviso in modo proporzionato tra gli anni.
        elif diff_year > 0:
            
            #'start_year_days' calcola i giorni totali che intercorrono tra la data di inizio e la fine dell'anno medesimo.
            start_year_days = daystoend(tot_loan['disburse_time'][i].tz_localize(None), format='%Y%m%d') 
            
            #'end_year_days' calcola quanti sono i giorni da considerare nell'ultimo anno (quello di 'planned_expiration_time')
            end_year_days = tot_loan['tot_days'][i] - start_year_days-(365*(diff_year-1))
            
            #vengono aggiunti al dizionario sia nell'anno di inizio sia nell'anno finale l'amount corrispondente suddiviso per giorni
            results[start_year] += round(tot_loan['debitxday'][i] * start_year_days,2)
            results[end_year] += round(tot_loan['debitxday'][i] * end_year_days,2)
            
            #Nei casi in cui la differenza tra gli anni sia maggiore di 1 viene fatto un ciclo con range 1: diff_year -1 poiché vengono 
            #calcolati anche gli anni interi compresi tra quello iniziale e quello finale. 
            if diff_year > 1:
                for j in range (1,diff_year - 1):
                    results[start_year + j] += round(tot_loan['debitxday'][i] * 365,2)     
                    
                    
#NON VENGONO PRESE IN CONSIDERAZIONE LE DATE CON 'PLANNED_EXPIRATION_TIME' PRECEDENTI A 'DISBURSE_TIME'
#probabilmente ciò è dovuto ad un imputazione errata del dato

time: 3min 9s


Il dizionario risulta essere compilato come segue:
    

In [22]:
results

{2005: 102850.0,
 2006: 1376575.0,
 2007: 15446525.0,
 2008: 39423050.0,
 2009: 59689475.0,
 2010: 72609150.0,
 2011: 91622343.70000002,
 2012: 114069101.5200002,
 2013: 124902657.51999913,
 2014: 154056683.6200008,
 2015: 162220572.52999973,
 2016: 162652743.2699985,
 2017: 175198059.39000055,
 2018: 6496430.519999999}

time: 11 ms


# QUESTION 3)
-  Find the lenders that have funded at least twice

Per riuscire a calcolare quante volte ogni lenders abbia contribuito a un prestito, è stato necessario riutilizzare il dataframe normalizzato creato al punto 1.

Applicando una groupby settando lenders come parola chiave otteniamo un conto di quante volte ogni prestatore contribuisce ad un prestito

In [23]:
tot_lenders = norm_loans_lenders.groupby('lenders').size().reset_index()
tot_lenders.columns = ['lenders','count']
tot_lenders.head(10)

,lenders,count
0,000,40
1,00000,39
2,0002,70
3,00mike00,1
4,0101craign0101,71
5,0132575,4
6,0154884,4
7,0161130,3
8,0169713,4
9,0185429,2


time: 33.1 s


Successivamente, impongo una condizione 'count>1' in modo da ottenere una tabella che includa solo i lenders che hanno finanziato un prestito almeno 2 volte.

In [24]:
count_lenders = tot_lenders[tot_lenders['count']>1]
count_lenders.head(10)

,lenders,count
0,000,40
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
6,0154884,4
7,0161130,3
8,0169713,4
9,0185429,2
10,0197462,2


time: 137 ms


Coloro che hanno effettuato un prestito sono 1383799 e coloro che hanno effettuato più di 2 prestiti sono 901811 

In [25]:
#len(tot_lenders) = 1383799
#len(count_lenders) = 901811 

time: 688 µs


# QUESTION 4)
- For each country, compute how many loans have involved that country as borrowers

Prima di risolvere il quesito è stata necessaria una fase di pulizia dei dati del file 'loans_csv': ho deciso di utilizzare il 'country_code' al posto di 'country_name', per effettuare queste verifiche poichè mi consente di collegarmi alla tabella 'country_stats', in modo univoco.

La tabella 'country_stats' è stata dunque utilizzata come riferimento per le informazioni riguardanti le caratteristiche geografiche. 

Importo il dataframe 'country_stats' tramite pandas.

In [26]:
country_stats=pd.read_csv(r'./cs/country_stats.csv')
#country_stats.columns

time: 82.6 ms


In [27]:
#loans_csv.isna().any()
#loans_csv.isna().any().sum()
#loans_csv.columns

time: 516 µs


Noto che per quanto riguarda il dataframe 'loans.csv' l'unico 'country_code' mancante è quello riferito alla nazione Namibia.

In [28]:
loans_csv[['country_code','country_name']][loans_csv['country_code'].isnull()]

,country_code,country_name
82889,NaN,Namibia
156970,NaN,Namibia
598087,NaN,Namibia
684876,NaN,Namibia
971827,NaN,Namibia
1134818,NaN,Namibia
1214923,NaN,Namibia
1281022,NaN,Namibia
1415763,NaN,Namibia


time: 624 ms


Allo stesso modo, controllo anche il dataframe 'country_stats' e noto che anche al suo interno l'unico country_code mancante è quello
della Namibia.

In [29]:
country_stats[['country_name','country_code']][country_stats['country_code'].isnull()]

,country_name,country_code
115,Namibia,NaN


time: 14.2 ms


Decido quindi di imputare il codice stringa 'NB' riferito alla nazione Namibia per rimpiazzare i valori mancanti da entrambi i dataframe.

In [30]:
loans_csv['country_code'] = loans_csv['country_code'].fillna(value='NB')
country_stats['country_code'] = country_stats['country_code'].fillna(value='NB')

time: 207 ms


In [31]:
#Questi sono gli stessi codici utilizzati in precedenza come verifica: controllo che i valori mancanti siano stati 
#effettivamente rimpiazzati.

#loans_csv[['country_code','country_name']][loans_csv['country_code'].isnull()]
#country_stats[['country_name','country_code']][country_stats['country_code'].isnull()]

time: 450 µs


Successivamente, decido di controllare se ci sono dei paesi che compaiono solo in uno dei due dataframe e scopro che ci sono 5 paesi che compaiono in 'loans_csv' e non in 'stats_csv':
'GZ'= Gaza, 'VC'= Saint Vincent and the Grenadines, 'VI'= Virgin Island, 'VU'= Vanuatu, 'GU'=Guam.

In [32]:
check = []
for i in (loans_csv['country_code'].unique()):
    if i not in country_stats['country_code'].unique():
        check.append(i)
print(check)

['GZ', 'VC', 'VI', 'VU', 'GU']
time: 101 ms


Dal momento che in prima battuta ho pensato fosse meglio prendere come tabella di riferimento per le nazioni 'country_stats' decido di eliminare le righe corrispondenti a questi 5 paesi.

In [33]:
for i in check:
    loans_csv = loans_csv.drop(loans_csv.loc[loans_csv['country_code'] == i].index)

time: 24.4 s


Avendo risolto i vari problemi di pulizia del dataframe, decido di risolvere il quesito appplicando una groupby per 'country_code' al fine di calcolare il numero di richieste di prestito provenienti da ciascuna nazione e metto le informazioni all'interno di un nuovo dataframe.

In [34]:
borrowers = pd.DataFrame(loans_csv.groupby('country_code').size().reset_index().rename(columns={0:'borrowers'}))
borrowers.head(10)

,country_code,borrowers
0,AF,2337
1,AL,3075
2,AM,13952
3,AZ,10172
4,BA,608
5,BF,3489
6,BG,296
7,BI,1727
8,BJ,5946
9,BO,25250


time: 149 ms


# QUESTION 5)
- For each country, compute the overall amount of money borrowed

Al fine di rispondere correttamente ai prossimi quesiti è stato necessario fare un ragionamento a priori:
- ho considerato la variabile 'funded_amount' come l'ammontare di denaro preso in prestito da altri paesi (ovvero dei fondi)
- mentre ho considerato la variabile 'loan_amount' come l'ammontare che la nazione ha prestato ad altri.

Per rispondere al quesito 5) ho inteso i money borrowed, come quelli presi in presito e ho utilizzato quindi la variabile 'funded_amount'.


Ho applicato una groupby per paese e effettuato la somma della colonna 'funded_amount' rispetto al country_code e 
ho inserito tutte le informazioni all'interno di un dataframe.

In [35]:
money_borrowed = pd.DataFrame(loans_csv[['country_code','funded_amount']].groupby('country_code').sum()).reset_index()
money_borrowed.head(10)

,country_code,funded_amount
0,AF,1964650.0
1,AL,3992475.0
2,AM,20284675.0
3,AZ,14191350.0
4,BA,477250.0
5,BF,4042950.0
6,BG,375300.0
7,BI,5119300.0
8,BJ,3829925.0
9,BO,41814950.0


time: 201 ms


# QUESTION 6)
- Like the previous point, but expressed as a percentage of the overall amount lent.

Come per il punto precedente (dopo aver deciso che il totale 'lent' si riferisse alla colonna 'loan_amount' e quindi 
ai soldi che i paesi hanno prestato ad altri), decido di calcolare tramite una grupby e una somma  il totale dei prestiti per paese.

In [36]:
money_lent = pd.DataFrame(loans_csv[['country_code','loan_amount']].groupby('country_code').sum()).reset_index()
money_lent.head(10)

,country_code,loan_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
5,BF,4085200.0
6,BG,375300.0
7,BI,5233450.0
8,BJ,3865825.0
9,BO,44226725.0


time: 139 ms


Successivamente, per calcolare la percentuale di ammontare dei prestiti che hanno coinvolto ciascuna nazione come richiedente, sull'ammontare che l'ha coinvolta come prestatrice ho creato un nuovo dataframe che mostrasse le variabili 'country_code', 'funded amount', 'loan_amount' e 'percentage'. 
La percentuale è stata calcolata dividendo il 'funded amount' per il 'loan amount' *100. 

Quando la percentuale si avvicina a 100, significa che il paese ha donato tanto quanto ha preso in prestito.

In [37]:
perc = pd.DataFrame(columns=['country_code','funded_amount','loan_amount','percentage'])
perc['country_code'] = money_borrowed['country_code']
perc['funded_amount'] = money_borrowed['funded_amount']
perc['loan_amount'] = money_lent['loan_amount']
perc['percentage'] = round(money_borrowed['funded_amount']/money_lent['loan_amount']*100,2)
perc.head(10)

,country_code,funded_amount,loan_amount,percentage
0,AF,1964650.0,1967950.0,99.83
1,AL,3992475.0,4307350.0,92.69
2,AM,20284675.0,22950475.0,88.38
3,AZ,14191350.0,14784625.0,95.99
4,BA,477250.0,477250.0,100.00
5,BF,4042950.0,4085200.0,98.97
6,BG,375300.0,375300.0,100.00
7,BI,5119300.0,5233450.0,97.82
8,BJ,3829925.0,3865825.0,99.07
9,BO,41814950.0,44226725.0,94.55


time: 32.1 ms


# QUESTION 7)
- Like the three previous points, but split for each year (with respect to disburse time).

Ciò che risulta necessario fare per risolvere questo quesito, prima di ogni altro passaggio è quello di creare una variabile 'year' ricavata dalla colonna 'disburse_time' della tabella loans_csv.
Ho utilizzato la funzione di pandas Pandas Series.dt.year.

In [38]:
loans_csv['year'] = data1.dt.year

time: 204 ms


RIFERITO AL PUNTO 4)

Applico una groupby per calcolare la numerosità per 'country_code' e 'year' e creo un nuovo dataframe che mi mostri i risultati.

In [39]:
borrowers1 = pd.DataFrame(loans_csv.groupby(['country_code','year']).size()).rename(columns={0:'borrowers'})
borrowers1.head(10)

borrowers
country_code year             
AF           2007.0        408
             2008.0        370
             2009.0        678
             2010.0        632
             2011.0        247
             2015.0          1
             2016.0          1
AL           2012.0        332
             2013.0        507
             2014.0        603

time: 607 ms


RISPETTO AL PUNTO 5)

Applico una groupby per 'country_code' e'year' per calcolare la somma dei finanziamenti ricevuti per nazione e per anno. Creo un nuovo dataframe che mi mostri i risultati.

In [40]:
split_year = pd.DataFrame(loans_csv[['country_code','funded_amount','year']].groupby(['country_code','year']).sum())
split_year.head(10)

funded_amount
country_code year                 
AF           2007.0       194975.0
             2008.0       365375.0
             2009.0       581825.0
             2010.0       563350.0
             2011.0       245125.0
             2015.0         6000.0
             2016.0         8000.0
AL           2012.0       411775.0
             2013.0       780850.0
             2014.0       875225.0

time: 193 ms


RIFERITO AL PUNTO 6)

Infine, creo un nuovo dataframe che mi mostri le percentuali di ogni paese, divise per anno, tra la quantità presa in prestito 'funded_amount e quella prestata 'loan_amount'.

In [41]:
percxyear = pd.DataFrame((loans_csv.groupby(['country_code','year'])['funded_amount'].sum())
                     /(loans_csv['loan_amount'].groupby(loans_csv['country_code']).sum())*100).rename(columns={0:'percentage'})
percxyear.head(10)

percentage
country_code year              
AF           2007.0    9.907518
             2008.0   18.566275
             2009.0   29.565030
             2010.0   28.626235
             2011.0   12.455855
             2015.0    0.304886
             2016.0    0.406514
AL           2012.0    9.559822
             2013.0   18.128316
             2014.0   20.319338

time: 287 ms


# QUESTION 8)
- For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

Per imputare a ciascun lender l'ammontare procapite di prestito in cui è stato coinvolto, assumo che ciascuno contribuisca egualmente; per prima cosa, decido di prendere come riferimento il dataframe 'norm_loans_lenders' creato al punto 1) e di eseguire una groupby per 'loan_id'.

In [42]:
start=pd.DataFrame(norm_loans_lenders.groupby(norm_loans_lenders['loan_id']).size()).reset_index().rename(columns={0:'count'})  

time: 2.47 s


Successivamente eseguo una merge per aggiungere a ciascuna riga loan_id-lender, una colonna con il numero di lender che partecipano al prestito considerato

In [43]:
norm_loans_lenders=pd.merge(norm_loans_lenders,start)

time: 30 s


Ora eseguo una merge con il dataframe 'loans_csv' per aggiungere la colonna 'loan_amount'

In [44]:
norm_loans_lenders=pd.merge(norm_loans_lenders,loans_csv[['loan_id','loan_amount']])

time: 10.2 s


Infine creo una nuova variabile 'procapita_amount' ottenuta dividendo il valore della variabile 'loan_amount' per il numero di lenders che partecipano al prestito ('count').

In [45]:
norm_loans_lenders['procapita_amount']=norm_loans_lenders['loan_amount']/norm_loans_lenders['count']

time: 334 ms


Eseguo una groupby per ottenere la quantità pro capite ('procapita_amount') di finanziamenti per ogni lender.

In [46]:
procapita_amount=round(norm_loans_lenders[['lenders','procapita_amount']].groupby('lenders').sum(),2)
procapita_amount.head(10)

,procapita_amount
lenders,
000,1764.29
00000,1380.69
0002,2472.56
00mike00,52.63
0101craign0101,2623.57
0132575,117.68
0154884,136.34
0161130,74.78
0169713,128.84


time: 20.1 s


# QUESTION 9)
- For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. 
Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one
computed from the rows where the country is known.

Poichè viene richiesta un'operazione simile a quella del punto 6 viene riutilizzato il dataset 'perc' creato nel medesimo.
Creo una nuova variabile con la differenza tra i 'money_lent' e i 'money_borrowed' per ciascuna nazione chiamata 'diff_fund_lent'.

In [47]:
perc['diff_fund_lent'] = round(money_lent['loan_amount']-money_borrowed['funded_amount'],2)
perc.head(10)

,country_code,funded_amount,loan_amount,percentage,diff_fund_lent
0,AF,1964650.0,1967950.0,99.83,3300.0
1,AL,3992475.0,4307350.0,92.69,314875.0
2,AM,20284675.0,22950475.0,88.38,2665800.0
3,AZ,14191350.0,14784625.0,95.99,593275.0
4,BA,477250.0,477250.0,100.00,0.0
5,BF,4042950.0,4085200.0,98.97,42250.0
6,BG,375300.0,375300.0,100.00,0.0
7,BI,5119300.0,5233450.0,97.82,114150.0
8,BJ,3829925.0,3865825.0,99.07,35900.0
9,BO,41814950.0,44226725.0,94.55,2411775.0


time: 70.6 ms


# QUESTION 10)

- Which country has the highest ratio between the difference computed at the previous point and the population?

Per risolvere questo quesito eseguo una merge tra il dataframe 'perc' e il dataframe 'country_stats' su 'country_code'

In [48]:
highest_ratio = perc.merge(country_stats, on='country_code')

time: 52.3 ms


Viene calcolato il rapporto tra la differenza 'diff_fund_lent' della Question 9) e la numerosità della popolazione.
Dall'output si può identificare la nazione con il rapporto più elevato

In [57]:
highest_ratio['ratio'] = highest_ratio['diff_fund_lent'] / highest_ratio['population']
#highest_ratio.iloc[highest_ratio['ratio'].idxmax()]

country_code                              WS
funded_amount                    1.15082e+07
loan_amount                      1.22122e+07
percentage                             94.24
diff_fund_lent                        703925
country_name                           Samoa
country_code3                            WSM
continent                            Oceania
region                             Polynesia
population                            196440
population_below_poverty_line            NaN
hdi                                    0.702
life_expectancy                          NaN
expected_years_of_schooling              NaN
mean_years_of_schooling                  NaN
gni                                      NaN
kiva_country_name                      Samoa
ratio                                3.58341
pop_up_poverty                   4.10463e+06
ratio2                              0.171495
Name: 85, dtype: object

time: 8.89 ms


In [50]:
print('Nazione con ratio massima:','\n',
      'Nazione =',highest_ratio['country_name'][highest_ratio['ratio'].idxmax()],
      '|','Country Code =',highest_ratio['country_code'][highest_ratio['ratio'].idxmax()],
      '|','Ratio =',round(highest_ratio['ratio'][highest_ratio['ratio'].idxmax()],3))

Nazione con ratio massima: 
 Nazione = Samoa | Country Code = WS | Ratio = 3.583
time: 6.31 ms


# QUESTION 11)
- Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

Il quesito 11) è simile al quesito nel punto 10) quindi riutilizzo lo stesso dataframe 'highest ratio'

Quello che segue è il calcolo delle persone che vivono sopra la soglia di povertà ('pop_up_poverty')

In [51]:
highest_ratio['pop_up_poverty'] = round(country_stats['population']-(country_stats['population']*country_stats['population_below_poverty_line']/100))

time: 18.9 ms


Calcolo il rapporto richiesto per la risoluzione dell'esercizio

In [52]:
highest_ratio['ratio2'] = highest_ratio['diff_fund_lent'] / highest_ratio['pop_up_poverty']

time: 4.68 ms


visualizzo la nazione con il massimo rapporto (considerando la popolazione sopra la poverty_line)

In [56]:
#highest_ratio[['country_name']].head(10)
#highest_ratio.iloc[highest_ratio['ratio2'].idxmax()]

country_code                                   US
funded_amount                         3.63653e+07
loan_amount                            4.6352e+07
percentage                                  78.45
diff_fund_lent                        9.98666e+06
country_name                        United States
country_code3                                 USA
continent                                Americas
region                           Northern America
population                              324459463
population_below_poverty_line                15.1
hdi                                      0.919553
life_expectancy                            79.222
expected_years_of_schooling               16.5382
mean_years_of_schooling                   13.2179
gni                                       53245.1
kiva_country_name                   United States
ratio                                   0.0307794
pop_up_poverty                        4.15198e+06
ratio2                                    2.40528


time: 17.9 ms


In [54]:
print('Nazione con il massimo rapporto (considerando la popolazione sopra la poverty_line):','\n',
      'Nazione =',highest_ratio['country_name'][highest_ratio['ratio2'].idxmax()],
      '|','Country Code =',highest_ratio['country_code'][highest_ratio['ratio2'].idxmax()],
      '|','Ratio =',round(highest_ratio['ratio2'][highest_ratio['ratio2'].idxmax()],3))

Nazione con il massimo rapporto (considerando la popolazione sopra la poverty_line): 
 Nazione = United States | Country Code = US | Ratio = 2.405
time: 6.62 ms


In [55]:
finish=dt.datetime.now()
print('Tempo di esecuzione totale: ',finish-initial)

Tempo di esecuzione totale:  0:07:16.498492
time: 1.93 ms
